In [1]:
import numpy as np
import math

## Reference Data

## Run Data

In [2]:
def better_mask_for(a, b):
    sa = np.sum(a, axis=-1)
    sb = np.sum(b, axis=-1)
    return sa > sb

def max_diff_for(a, b):
    sa = np.sum(a, axis=-1)
    sb = np.sum(b, axis=-1)
    
    diff = sa-sb
    min_diff = np.min(diff)
    max_diff = np.max(diff)
    
    min_pos = np.amin(diff)
    max_pos = np.amax(diff)
    
    print("min", min_diff, "max", max_diff)
    
    return diff.astype("float"), min_diff, max_diff, min_pos, max_pos

# which of the above graph's points protrude further
def maximum_qs(qs_list):
    best = np.zeros_like(qs_list[0])
    
    for qs in qs_list:
        better_mask = better_mask_for(qs, best)
        best[better_mask] = qs[better_mask]
        
    return best

In [3]:
import glob, re
import itertools, operator

### All Graphs

### Cat Codes

## Plot

In [4]:
data_ref = np.load("./volumes/single-letter-best.npz")
qs_ref = data_ref["qs"]

In [209]:
f = glob.glob("./volumes/concat_codes/*1-in-50.*-best.npz")[0]
print(f)
data_comp = np.load(f)
qs_comp = data_comp["qs"]

./volumes/concat_codes/concatcodes.Pj5zcGFyc2U2PDw6cl9PR0NBQD9fT0dDQUA-X09HQ0FAP19PR0NBQD9fT0dDQUA-X09HQ0FAP19PR0NBQD9fT0dDQUA-X09Ofn5+fgo=.1-in-50.json.gz.tar-best.npz


In [210]:
diff, min_diff, max_diff, min_pos, max_pos = max_diff_for(qs_comp, qs_ref)

min -0.11868817364767964 max 0.15977982400045065


In [211]:
from matplotlib import pyplot


# nonlinear color transformation around midpoint
from matplotlib.colors import SymLogNorm
color_mapper = SymLogNorm(linthresh=.00008, linscale=.00008, vmin=-.1, vmax=.1)

from matplotlib import cm
viridis = cm.get_cmap('Spectral')

In [212]:
import mayavi
import mayavi.mlab as mlab

In [213]:
x,y,z = qs_comp.transpose()

In [214]:
dxx, dyy, dzz, dds = [ a.reshape(int(np.sqrt(len(x))), -1) for a in (x, y, z, diff) ]

In [215]:
ANTIALIAS = 8
LINEWIDTH = 2

mlab.options.offscreen = True
mlab.figure(bgcolor=(1,1,1), size=(1024,1024))

mesh = mlab.mesh(dxx, dyy, dzz, scalars=dds, colormap="Spectral", opacity=.95)
mesh.actor.property.lighting = False
mesh.module_manager.scalar_lut_manager.lut.table = 255*viridis(color_mapper(np.linspace(-.1, .1, 255)))
mesh.module_manager.scalar_lut_manager.data_range = (-.1, .1)

AXES_RANGE = .52
mlab.plot3d((0,AXES_RANGE),(0,0),(0,0),line_width=LINEWIDTH*ANTIALIAS,tube_radius=None, color=(0,0,0))
mlab.plot3d((0,0),(0,AXES_RANGE),(0,0),line_width=LINEWIDTH*ANTIALIAS,tube_radius=None, color=(0,0,0))
mlab.plot3d((0,0),(0,0),(0,AXES_RANGE),line_width=LINEWIDTH*ANTIALIAS,tube_radius=None, color=(0,0,0))

for i in np.arange(.1, .4, .1):
    #bottom
    mlab.plot3d((i,i),(0,AXES_RANGE),(0,0),line_width=LINEWIDTH*ANTIALIAS,tube_radius=None, color=(.75,.75,.75))
    mlab.plot3d((0,AXES_RANGE),(i,i),(0,0),line_width=LINEWIDTH*ANTIALIAS,tube_radius=None, color=(.75,.75,.75))
    #left
    mlab.plot3d((i,i),(0,0),(0,AXES_RANGE),line_width=LINEWIDTH*ANTIALIAS,tube_radius=None, color=(.75,.75,.75))
    mlab.plot3d((0,AXES_RANGE),(0,0),(i,i),line_width=LINEWIDTH*ANTIALIAS,tube_radius=None, color=(.75,.75,.75))
    #right
    mlab.plot3d((0,0),(i,i),(0,AXES_RANGE),line_width=LINEWIDTH*ANTIALIAS,tube_radius=None, color=(.75,.75,.75))
    mlab.plot3d((0,0),(0,AXES_RANGE),(i,i),line_width=LINEWIDTH*ANTIALIAS,tube_radius=None, color=(.75,.75,.75))


mlab.plot3d((.5,.5),(0,AXES_RANGE),(0,0),line_width=LINEWIDTH*ANTIALIAS,tube_radius=None, color=(.5,.5,.5))
mlab.plot3d((0,AXES_RANGE),(.5,.5),(0,0),line_width=LINEWIDTH*ANTIALIAS,tube_radius=None, color=(.5,.5,.5))

mlab.plot3d((.5,.5),(0,0),(0,AXES_RANGE),line_width=LINEWIDTH*ANTIALIAS,tube_radius=None, color=(.5,.5,.5))
mlab.plot3d((0,AXES_RANGE),(0,0),(.5,.5),line_width=LINEWIDTH*ANTIALIAS,tube_radius=None, color=(.5,.5,.5))

mlab.plot3d((0,0),(.5,.5),(0,AXES_RANGE),line_width=LINEWIDTH*ANTIALIAS,tube_radius=None, color=(.5,.5,.5))
mlab.plot3d((0,0),(0,AXES_RANGE),(.5,.5),line_width=LINEWIDTH*ANTIALIAS,tube_radius=None, color=(.5,.5,.5))

#colorbar = mlab.colorbar(orientation='vertical', nb_labels=9)
#colorbar.label_text_property.font_family = 'courier'
#colorbar.label_text_property.bold = 0
#colorbar.label_text_property.font_size=24
#colorbar.label_text_property.color = (0,0,0)

scene = mlab.gcf().scene
scene.parallel_projection = True

In [216]:
mlab.savefig("/home/jkrb2/Desktop/snapshot.png", magnification=ANTIALIAS)